In [2]:
# Snowpark Libs
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.functions import col,to_date, udf
# Data Science Libs
import pandas as pd
import os
import dotenv
from pathlib import Path
from datetime import datetime
import numpy as np
import datetime
import pandas as pd



# Laod packages

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import Parallel, delayed

In [3]:
# Read config parser .ini file with your connection information

Path = r"/Users/chitrakumarsaichv/Library/CloudStorage/OneDrive-kochind.com/Desktop/Projects/purchasing_anomaly/.env"
dotenv.load_dotenv(Path)

# SSO - KochID
connection_parms = {
    "account": os.getenv('account'),
    "user": os.getenv('email'),
    "authenticator": os.getenv('authenticator'),
    "database": os.getenv('database'),
    "schema": os.getenv('schema'),
    "warehouse": os.getenv('warehouse'),"role": os.getenv('role')
} 


# Create Snowflake Session object
session = Session.builder.configs(connection_parms).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://auth.kochid.com/idp/SSO.saml2?SAMLRequest=nZLBUtswEIZfxaOeLcmGdFpNEsYlUNIhxIkDDNyEpSRqZMnVynF4%2ByoOnqEHOPTmkf9dfdpvhxeHSkd76UBZM0IJpiiSprRCmc0I3a%2Bu428oAs%2BN4NoaOUKvEtDFeAi80jXLGr81S%2FmnkeCj0MgA636MUOMMsxwUMMMrCcyXrMhmtyzFlNXOeltajd6VfF7BAaTzgbAvEaAC3tb7mhHSti1uz7B1G5JSSgn9TkLqGPnS5w%2FhTR%2FkE0LPj%2FmQCPH8je2HMqcRfIb1cgoBu1mt8jifFysUZT3qpTXQVNIV0u1VKe%2BXtycACATK7FUYahiEwA3EkoOPU8xbwGBsu9Z8J0tb1Y0P3XH4ImspiLYbFQYwnYxQvVNCX5nF7%2FXzblZebedqevA3T7PHLeelfjhPfi2WhXx6WTz%2BzOkgK1H00BtOj4anAI2cmqNXH45oOohpGidnq2TAaMoGFH8d0GcUTYJXZbjvKnt4HqTjnS23SnRsStSkKOb4qDFFp9Vg3Q1u%2FL8PHpL3Xd627S4ImE5yq1X5Gl1bV3H%2FsZ8EJ92JEvG6izJZcaUzIZwECJ60tu2lk9yHpfaukYiMT7f%2Bu9bjvw%3D%3D&RelayState=ver%3A1-hint%3A759822674653786-ETMsDgAAAZT%2F1YQaABRBRVMvQ0

You might have more than one threads sharing the Session object trying to update sql_simplifier_enabled. Updating this while other tasks are running can potentially cause unexpected behavior. Please update the session configuration before starting the threads.


In [4]:
def get_data(session: Session) -> pd.DataFrame:
    # Load data from Snowflake
    """
    This function loads data from the table PURCHASES in the Snowflake database

    Args:
        session (Session): snowflake session object

    Returns:
        pd.DataFrame: returns the data from the table PURCHASES
    """
    with open("/Users/chitrakumarsaichv/Library/CloudStorage/OneDrive-kochind.com/Desktop/Projects/purchasing_anomaly/data/purchasing_anomaly.sql", "r") as file:
        query = file.read()
    df = pd.read_sql(query, session.connection)
    return df

In [5]:
df = get_data(session)

/var/folders/d6/zg1ft6rd1s7122t4rmw9kkpr0000gn/T/ipykernel_78558/3770731770.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, session.connection)


In [6]:
df.head()

,Purchase Doc Date,Purchase Doc Month,Purchase Doc Quarter,Purchase Doc Year,Requested Delivery Date,Vendor ID,Vendor NM,Plant Desc,Purchase Doc ID,Purchase Doc Item ID,...,Cost Center Category Desc,Work Order No,Network ID,Project ID,Currency Desc,Long Text,Purchase Doc Item Ct,PO Qty,PO Item Net Order Value USD,User Desc
0,2025-02-12,2025-02,2025 QTR-1,2025,2025-02-21,8010005836,FASTENAL CANADA LTD,Kingston (CA) MFG,2000137958,300,...,Maintenance,,,NaN,Canadian Dollar,None,1,25.0,4.512,"Louwerse, James"
1,2025-02-12,2025-02,2025 QTR-1,2025,2025-02-21,8010005836,FASTENAL CANADA LTD,Kingston (CA) MFG,2000137958,440,...,Maintenance,,,NaN,Canadian Dollar,None,1,25.0,4.685,"Louwerse, James"
2,2025-02-12,2025-02,2025 QTR-1,2025,2025-02-21,8010005836,FASTENAL CANADA LTD,Kingston (CA) MFG,2000137958,600,...,Maintenance,,,NaN,Canadian Dollar,None,1,25.0,13.015,"Louwerse, James"
3,2025-02-12,2025-02,2025 QTR-1,2025,2025-02-21,8010005836,FASTENAL CANADA LTD,Kingston (CA) MFG,2000137996,140,...,Maintenance,,,NaN,Canadian Dollar,None,1,25.0,2.776,"Louwerse, James"
4,2025-02-12,2025-02,2025 QTR-1,2025,2025-02-13,8010000984,GRAINGER INC,Camden Polymer (US) MFG,2000137997,10,...,Total Production,,,NaN,US Dollar,None,1,1.0,9.070,"Anderson, Jonathan"


In [7]:
# Define stopwords manually (Scikit-learn's stopword list is English only)
stopwords = set([
    'in', 'height', 'size', 'length', 'mil', 'width', 'ft', 
    'lb', 'overall', 'yes', 'can', 'for', 'per', 'and',
    'the', 'wt', 'oz', 'of', 'or', 'a', 'do', 'an',
    'any', 'am', 'all', 'after', 'again', 'above', 'by',
    'should', 'did', 'each', 'had', 'from', 'her', 'him',
    'but', 'by', 'below', 'between', 'at', 'such', 'some',
    'same', 'out', 'with', 'and', 'is', 'so', 'staples'
])

def process_data(df: pd.DataFrame) -> pd.DataFrame:
    """Filter dataset to keep only Purchase Orders starting with '55' or '20'."""
    df = df[df['Purchase Doc ID'].astype(str).str[:2].isin(['55', '20'])].reset_index(drop=True)
    return df

def get_date_features(df: pd.DataFrame, column_name: str):
    """Extract year and month from a date column."""
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
    df[f'{column_name} Year'] = df[column_name].dt.year
    df[f'{column_name} Month'] = df[column_name].dt.month
    return df

def clean_text(text_series: pd.Series) -> pd.Series:
    """Cleans text: removes stopwords, digits, and short words."""
    text_series_clean = text_series.str.lower()
    text_series_clean = text_series_clean.str.replace(r'[^a-z\s]', '', regex=True)  # Remove non-alphabetic characters
    text_series_clean = text_series_clean.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2 and w not in stopwords]))
    return text_series_clean

def get_unique_words(df: pd.DataFrame, clean_text_col: str) -> list:
    """Returns unique words from the clean text column."""
    all_words = df[clean_text_col].str.split().explode().unique()
    return list(set(all_words))

def first_element(text: str) -> str:
    """Extracts the first meaningful part of a text (for vendor-specific processing)."""
    return text.split(', ')[0].split(' - ')[0].split('---')[0] if isinstance(text, str) else ''

def get_short_text_by_vendor(df: pd.DataFrame) -> pd.DataFrame:
    """Extracts the relevant short text for each vendor."""
    df['text_clean'] = df['Purchase Document Item Short Text'].copy()  # Default to short text

    for i in df.index:
        long_text = df.loc[i, 'Long Text']  # Use the correct column name

        if pd.isna(long_text):  # If Long Text is NaN, use Short Text
            df.loc[i, 'text_clean'] = df.loc[i, 'Purchase Document Item Short Text']
        elif df.loc[i, 'Vendor NM'] == 'GRAINGER INC':
            df.loc[i, 'text_clean'] = first_element(long_text)
        elif df.loc[i, 'Vendor NM'] == 'AMAZON.COM SERVICES INC':
            df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
        elif df.loc[i, 'Vendor NM'] == 'STAPLES ADVANTAGE':
            df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
            product_text = str(long_text).split(', ')[0]
            df.loc[i, 'text_clean'] = product_text.split('---')[0]
    
    return df


def extract_nouns_tfidf(text_series: pd.Series) -> list:
    """Uses TF-IDF Vectorizer to extract key words for each row (approximate nouns)."""
    vectorizer = TfidfVectorizer(stop_words="english", max_features=min(len(text_series), 1000))  # Adjust feature limit dynamically
    X = vectorizer.fit_transform(text_series)

    # Convert TF-IDF matrix back to text
    feature_names = vectorizer.get_feature_names_out()

    # Extract top words per row
    word_lists = []
    for row in X.toarray():
        top_indices = row.argsort()[-5:][::-1]  # Get top 5 words per row
        top_words = [feature_names[i] for i in top_indices if row[i] > 0]
        word_lists.append(" ".join(top_words))

    return word_lists

def extract_noun_words(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
    """Extracts top words using TF-IDF as a proxy for important words."""
    df['noun_words'] = extract_nouns_tfidf(df[text_col])
    return df

def get_noun_words_reduced(df: pd.DataFrame):
    """Filters clean text to retain only top 5 extracted nouns."""
    df['text_clean'] = df['text_clean'].str.split()
    df['noun_words'] = df['noun_words'].str.split()
    
    df['noun_words_reduced'] = df.apply(lambda row: ' '.join([w for w in row['text_clean'][:5] if w in row['noun_words']]), axis=1)
    
    df['text_clean'] = df['text_clean'].str.join(' ')
    df['noun_words'] = df['noun_words'].str.join(' ')
    
    return df

def text_parallel_clean(array):
    """Parallel processing wrapper for extract_nouns_tfidf."""
    result = Parallel(n_jobs=8, backend="multiprocessing")(delayed(extract_nouns_tfidf)(text) for text in array)
    return result

def series_of_lists_to_list_of_strings(series_of_lists: pd.Series) -> list:
    """Converts lists in a Pandas series to a list of joined strings."""
    return series_of_lists.apply(lambda x: ' '.join(x) if isinstance(x, list) else '')

def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Formats column names for consistency."""
    df.columns = df.columns.str.replace('_', ' ').str.title()
    return df

In [8]:
df = process_data(df)
df = get_short_text_by_vendor(df)
df['text_clean'] = clean_text(df['text_clean'])
df = extract_noun_words(df, "text_clean")
df = get_noun_words_reduced(df)
df = clean_cols(df)

In [9]:
# Ensure 'Purchase Doc Date' is in datetime format
df['Purchase Doc Date'] = pd.to_datetime(df['Purchase Doc Date'], errors='coerce')

# Sort by 'Purchase Doc Date' in ascending order (oldest to newest)
df = df.sort_values(by='Purchase Doc Date', ascending=True).reset_index(drop=True)

In [10]:
df[['Purchase Doc Date','Purchase Document Item Short Text', 'Noun Words Reduced', 'Noun Words',"Plant Desc"]].tail(50)

,Purchase Doc Date,Purchase Document Item Short Text,Noun Words Reduced,Noun Words,Plant Desc
950,2025-01-27,"15/16"" x 6.000"" OAL Chrome Forged Alloy",oal chrome alloy,chrome alloy oal,Kingston (CA) MFG
951,2025-01-27,"9.1"" x 12.5"" CC500 White Low Lint Perfor",white low lint perfor,perfor lint low white,Kingston (CA) MFG
952,2025-01-27,TK116384707T Socket Extension Set Output,tkt socket extension set output,output extension socket set tkt,Victoria (US) MFG
953,2025-01-28,"Staples Graph Pad, 11"" x 17"", Graph Rule",graph pad graph,graph pad,Victoria (US) MFG
954,2025-01-28,Surge Protector Power Strip - Nuetsa Fla,surge protector power strip,strip surge fla protector power,Victoria (US) MFG
955,2025-01-28,M Beige Premium Grain Cowhide Unlined Sl,beige premium grain cowhide unlined,unlined beige cowhide premium grain,Kingston (CA) MFG
956,2025-01-28,CRAFTSMAN CMHT82298 CFT MINI DIAGONAL PL,craftsman mini,craftsman mini,Victoria (US) MFG
957,2025-01-29,"1/2"" x 1/2"" OAL Square Socket Adapter",oal square socket adapter,adapter square oal socket,Kingston (CA) MFG
958,2025-01-30,Swingline Ultra Economy Pro Desk Stapler,swingline ultra pro desk,swingline stapler desk pro ultra,Kingston (CA) MFG
959,2025-01-31,"Runstar Smart Scale FSA or HSA Eligible,",smart scale,scale smart,Victoria (US) MFG


In [11]:
def get_last_week_data_for_each_plant(df: pd.DataFrame) -> pd.DataFrame:
    """Filters data to keep only the last week of data for each plant."""
    df = df.groupby('Plant Desc').apply(lambda x: x.nlargest(7, 'Purchase Doc Date')).reset_index(drop=True)
    return df

In [12]:
get_last_week_data_for_each_plant(df)

/var/folders/d6/zg1ft6rd1s7122t4rmw9kkpr0000gn/T/ipykernel_78558/1210642113.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Plant Desc').apply(lambda x: x.nlargest(7, 'Purchase Doc Date')).reset_index(drop=True)


,Purchase Doc Date,Purchase Doc Month,Purchase Doc Quarter,Purchase Doc Year,Requested Delivery Date,Vendor Id,Vendor Nm,Plant Desc,Purchase Doc Id,Purchase Doc Item Id,...,Project Id,Currency Desc,Long Text,Purchase Doc Item Ct,Po Qty,Po Item Net Order Value Usd,User Desc,Text Clean,Noun Words,Noun Words Reduced
0,2025-02-12,2025-02,2025 QTR-1,2025,2025-02-13,8010000984,GRAINGER INC,Camden Polymer (US) MFG,2000137997,10,...,NaN,US Dollar,None,1,1.0,9.070,"Anderson, Jonathan",tkt insulating electrical tape,insulating electrical tape tkt,tkt insulating electrical tape
1,2025-02-10,2025-02,2025 QTR-1,2025,2025-02-17,8010000984,GRAINGER INC,Camden Polymer (US) MFG,2000137618,80,...,NaN,US Dollar,None,1,1.0,24.440,"Brown, Obed",tkt hand sanitizer form gel,form gel hand tkt,tkt hand form gel
2,2025-02-05,2025-02,2025 QTR-1,2025,2025-02-06,8010099066,AMAZON.COM SERVICES INC,Camden Polymer (US) MFG,2000137097,30,...,NaN,US Dollar,None,1,1.0,118.990,"Bartlett, Wayne",olight arkfeld ultra lumens edc fla,lumens olight fla edc ultra,olight ultra lumens edc
3,2025-02-05,2025-02,2025 QTR-1,2025,2025-02-07,8010008932,STAPLES ADVANTAGE,Camden Polymer (US) MFG,2000137093,20,...,NaN,US Dollar,None,1,1.0,80.550,"Watkins, Cristin",coffee mate french vanilla powdered crea,powdered mate vanilla coffee,coffee mate vanilla powdered
4,2025-02-05,2025-02,2025 QTR-1,2025,2025-02-14,8010099066,AMAZON.COM SERVICES INC,Camden Polymer (US) MFG,2000137149,20,...,NaN,US Dollar,None,1,1.0,129.000,"Arledge, John",arrowmax ahdhbkm noise cancelling,noise,noise
5,2025-01-22,2025-01,2025 QTR-1,2025,2025-01-23,8010000984,GRAINGER INC,Camden Polymer (US) MFG,2000135729,20,...,NaN,US Dollar,None,1,1.0,25.190,"Robinson, B",tkt safety glasses series khor,glasses safety series tkt,tkt safety glasses series
6,2025-01-21,2025-01,2025 QTR-1,2025,2025-01-24,8010099066,AMAZON.COM SERVICES INC,Camden Polymer (US) MFG,2000135593,20,...,NaN,US Dollar,None,1,1.0,30.850,"Lipsey, Cynthia",original donut shop regular keurig,regular donut shop keurig original,original donut shop regular keurig
7,2025-02-10,2025-02,2025 QTR-1,2025,2025-02-12,8010008932,STAPLES ADVANTAGE,Houston (US) MFG,2000137537,70,...,NaN,US Dollar,None,1,1.0,33.240,"Smith, Maritza",dunkin original blend coffee keurig,blend dunkin keurig original coffee,dunkin original blend coffee keurig
8,2025-02-07,2025-02,2025 QTR-1,2025,2025-02-14,8010000984,GRAINGER INC,Houston (US) MFG,2000137408,50,...,NaN,US Dollar,None,1,6.0,121.980,"Reid, Ridge",tkt safety glasses series dawso,glasses safety series tkt,tkt safety glasses series
9,2025-02-07,2025-02,2025 QTR-1,2025,2025-02-10,8010099066,AMAZON.COM SERVICES INC,Houston (US) MFG,2000137389,30,...,NaN,US Dollar,None,1,1.0,14.990,"Reid, Ridge",sandisk extreme pro sdxc uhsi memo,pro,pro


In [13]:
df.shape

(1000, 40)

In [14]:
import re

df.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                         col.replace(' ', "_")).upper() for col in df.columns]

In [15]:
df.head()

,PURCHASE_DOC_DATE,PURCHASE_DOC_MONTH,PURCHASE_DOC_QUARTER,PURCHASE_DOC_YEAR,REQUESTED_DELIVERY_DATE,VENDOR_ID,VENDOR_NM,PLANT_DESC,PURCHASE_DOC_ID,PURCHASE_DOC_ITEM_ID,...,PROJECT_ID,CURRENCY_DESC,LONG_TEXT,PURCHASE_DOC_ITEM_CT,PO_QTY,PO_ITEM_NET_ORDER_VALUE_USD,USER_DESC,TEXT_CLEAN,NOUN_WORDS,NOUN_WORDS_REDUCED
0,2023-11-22,2023-11,2023 QTR-4,2023,2023-11-23,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,2000091638,70,...,NaN,Canadian Dollar,None,1,4.0,7.554,"Stephenson, Michael",cell duracellreg procellreg,procellreg duracellreg cell,cell duracellreg procellreg
1,2023-11-22,2023-11,2023 QTR-4,2023,2023-11-24,8010100205,AMAZON BUSINESS CANADA,Kingston (CA) MFG,2000091652,30,...,NaN,Canadian Dollar,None,1,1.0,7.952,"Rogers, Carl",jetech camera lens protector iphone,camera protector lens iphone,camera lens protector iphone
2,2023-11-24,2023-11,2023 QTR-4,2023,2023-12-06,8010005836,FASTENAL CANADA LTD,Kingston (CA) MFG,2000091824,10,...,NaN,Canadian Dollar,None,1,8.0,157.974,"Miller, Andrew",medium mtm rugged comfo,mtm medium,medium mtm
3,2023-11-24,2023-11,2023 QTR-4,2023,2023-11-28,8010008932,STAPLES ADVANTAGE,Orange (US) MFG,2000091803,50,...,NaN,US Dollar,None,1,2.0,37.260,"Ross, Leslie",dixie ultra pathways heavyweight paper,pathways heavyweight dixie ultra paper,dixie ultra pathways heavyweight paper
4,2023-11-24,2023-11,2023 QTR-4,2023,2023-12-08,8010005836,FASTENAL CANADA LTD,Kingston (CA) MFG,2000091822,10,...,NaN,Canadian Dollar,None,1,6.0,49.363,"MacCrae, Cole",net fill aerosol equipment,net aerosol,net aerosol


In [16]:
df.columns

Index(['PURCHASE_DOC_DATE', 'PURCHASE_DOC_MONTH', 'PURCHASE_DOC_QUARTER',
       'PURCHASE_DOC_YEAR', 'REQUESTED_DELIVERY_DATE', 'VENDOR_ID',
       'VENDOR_NM', 'PLANT_DESC', 'PURCHASE_DOC_ID', 'PURCHASE_DOC_ITEM_ID',
       'BI_LAST_UPDATED', 'REQUISITIONER_ID', 'CREATED_BY',
       'REQUISITIONER_MASTER', 'PURCHASE_DOCUMENT_ITEM_SHORT_TEXT', 'UOM',
       'ACCOUNT_ASSIGNMENT_CATEGORY_ID', 'MATERIAL_ID', 'MATERIAL_GROUP_ID',
       'MATERIAL_GROUP_DESC', 'SUB_COMMODITY_DESC', 'COMMODITY_DESC',
       'GL_ACCOUNT_ID', 'GL_ACCOUNT_DESC', 'COST_CENTER_ID',
       'COST_CENTER_DESC', 'COST_CENTER_PERSON_RESP',
       'COST_CENTER_CATEGORY_DESC', 'WORK_ORDER_NO', 'NETWORK_ID',
       'PROJECT_ID', 'CURRENCY_DESC', 'LONG_TEXT', 'PURCHASE_DOC_ITEM_CT',
       'PO_QTY', 'PO_ITEM_NET_ORDER_VALUE_USD', 'USER_DESC', 'TEXT_CLEAN',
       'NOUN_WORDS', 'NOUN_WORDS_REDUCED'],
      dtype='object')

In [17]:
def get_word_categories(df: pd.DataFrame, use_annotations: bool=False, df_labels=None) -> pd.DataFrame:
    """Reduced noun_words will be split into common_words and rare_words.
    A unique word count is given to each record.

    Args:
        df (pd.DataFrame): dataframe with NOUN_WORDS_REDUCED column
        use_annotations (bool, optional): When use_annotations=True, then if red_flag = 1,
            then the words in the record are not considered in the corpus.
            Defaults to False.
        df_labels (pd.DataFrame, optional): df_labels data in s3. Defaults to None to pull labels from DynamoDB table.
    """
#     start_date = df['purchase_doc_date'].max() - pd.Timedelta(days=450)
#     df = df[df['purchase_doc_date']>start_date].copy()
#     df = df.merge(df_labels, on='tdname', how='left')

    df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].replace(np.nan, 'blank')
    df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].str.split()

    dff = pd.DataFrame()
    for plant in df['PLANT_DESC'].unique():
        print(f'Processing {plant}')

        df_plant = df[df['PLANT_DESC']==plant].copy()

        all_text = []
        if use_annotations:
            for i in df_plant[df_plant['annotation']!=1]['NOUN_WORDS_REDUCED']:
                all_text.extend(i)
        else:
            for i in df_plant['NOUN_WORDS_REDUCED']:
                all_text.extend(i)
            
        ###################
        text_series = pd.Series(all_text)
        rare_words = list(text_series.value_counts()[text_series.value_counts()<=3].index)
        rare_words_list = []
        common_words_list = []
        unique_count = []
        for ind in df_plant['NOUN_WORDS_REDUCED'].index:
            unique_counter = 0
            rare_words_found = []
            common_words_found = []
            for w in df_plant.loc[ind, 'NOUN_WORDS_REDUCED']:
                if w in rare_words:
                    unique_counter+=1
                    rare_words_found.append(w)
                else:
                    common_words_found.append(w)
            rw = ' '.join(rare_words_found)
            cw = ' '.join(common_words_found)
            rare_words_list.append(rw)
            common_words_list.append(cw)
            unique_count.append(unique_counter)
            
        df_plant['unique_count'] = unique_count
        df_plant['common_words'] = common_words_list
        df_plant['rare_words'] = rare_words_list
        df_plant.columns = df_plant.columns.str.upper()
        df_plant['NOUN_WORDS_REDUCED'] = df_plant['NOUN_WORDS_REDUCED'].apply(lambda x: ' '.join(x))

        dff = pd.concat([dff, df_plant], axis=0)
    return dff


In [18]:
get_word_categories(df)

Processing Maitland (CA)  MFG
Processing Kingston (CA) MFG
Processing Orange (US) MFG
Processing Victoria (US) MFG
Processing Camden Polymer (US) MFG
Processing Longview (US) MFG
Processing Seaford (US) MFG
Processing Houston (US) MFG


,PURCHASE_DOC_DATE,PURCHASE_DOC_MONTH,PURCHASE_DOC_QUARTER,PURCHASE_DOC_YEAR,REQUESTED_DELIVERY_DATE,VENDOR_ID,VENDOR_NM,PLANT_DESC,PURCHASE_DOC_ID,PURCHASE_DOC_ITEM_ID,...,PURCHASE_DOC_ITEM_CT,PO_QTY,PO_ITEM_NET_ORDER_VALUE_USD,USER_DESC,TEXT_CLEAN,NOUN_WORDS,NOUN_WORDS_REDUCED,UNIQUE_COUNT,COMMON_WORDS,RARE_WORDS
0,2023-11-22,2023-11,2023 QTR-4,2023,2023-11-23,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,2000091638,70,...,1,4.0,7.554,"Stephenson, Michael",cell duracellreg procellreg,procellreg duracellreg cell,cell duracellreg procellreg,1,cell duracellreg,procellreg
20,2023-11-30,2023-11,2023 QTR-4,2023,2023-12-01,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,2000092331,160,...,1,3.0,24.530,None,tube stokolanreg white afterwo,tube white,tube white,1,white,tube
56,2023-12-14,2023-12,2023 QTR-4,2023,2023-12-18,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,5501700867,10,...,1,48.0,433.642,"MacDonald, Joanne",merv synthetic fiber,synthetic,synthetic,1,,synthetic
57,2023-12-14,2023-12,2023 QTR-4,2023,2023-12-15,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,2000093837,50,...,1,5.0,59.739,None,scape goat winter gloves linin,winter goat scape gloves,scape goat winter gloves,4,,scape goat winter gloves
58,2023-12-14,2023-12,2023 QTR-4,2023,2023-12-15,8010005836,FASTENAL CANADA LTD,Maitland (CA) MFG,2000093837,80,...,1,1.0,5.937,None,white fullgrain goatskin leather scap,scap fullgrain leather goatskin white,white fullgrain goatskin leather scap,4,white,fullgrain goatskin leather scap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920,2025-01-15,2025-01,2025 QTR-1,2025,2025-01-17,8010099066,AMAZON.COM SERVICES INC,Houston (US) MFG,2000134989,10,...,1,1.0,69.990,"Purdion Shute, Lexus",purrugs dirt trapper door mat,purrugs,purrugs,1,,purrugs
972,2025-02-05,2025-02,2025 QTR-1,2025,2025-02-12,8010008932,STAPLES ADVANTAGE,Houston (US) MFG,2000137161,10,...,1,1.0,13.990,"Macias, Daisy",engraved plastic sign cus,sign plastic,plastic sign,2,,plastic sign
978,2025-02-07,2025-02,2025 QTR-1,2025,2025-02-14,8010000984,GRAINGER INC,Houston (US) MFG,2000137408,50,...,1,6.0,121.980,"Reid, Ridge",tkt safety glasses series dawso,glasses safety series tkt,tkt safety glasses series,3,tkt,safety glasses series
979,2025-02-07,2025-02,2025 QTR-1,2025,2025-02-10,8010099066,AMAZON.COM SERVICES INC,Houston (US) MFG,2000137389,30,...,1,1.0,14.990,"Reid, Ridge",sandisk extreme pro sdxc uhsi memo,pro,pro,1,,pro


## Making it Snowflake Ready

In [30]:
def purchasing_anomaly(session: Session)  -> str:
    import pandas as pd
    import numpy as np
    from sklearn.feature_extraction.text import TfidfVectorizer
    from snowflake.snowpark import Session
    import re

    def process_data() -> pd.DataFrame:
        query = """
    SELECT 
        PAM.*, ZM."User Desc" 
    FROM 
        ZINT_PR.UI_ALL.VW_RPT_PURCHASE_ANOMALIES_MODEL AS PAM
    LEFT JOIN 
        (SELECT "User ID", "User Desc" 
            FROM "ZINT_MASTER"."UI_SEC"."AD Account") 
        AS ZM ON ZM."User ID" = PAM."Requisitioner Master"
    WHERE "Purchase Doc Date" >= DATEADD(DAY, -450, GETDATE())
"""

        df = pd.read_sql(query, session.connection)
        df = df[df['Purchase Doc ID'].astype(str).str[:2].isin(['55', '20'])].reset_index(drop=True)
        return df

    def get_date_features(df: pd.DataFrame, column_name: str):
        """Extract year and month from a date column."""
        df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
        df[f'{column_name} Year'] = df[column_name].dt.year
        df[f'{column_name} Month'] = df[column_name].dt.month
        return df

    def clean_text(text_series: pd.Series) -> pd.Series:
        """Cleans text: removes stopwords, digits, and short words."""
            # Define stopwords manually (Scikit-learn's stopword list is English only)
        stopwords = set([
            'in', 'height', 'size', 'length', 'mil', 'width', 'ft', 
            'lb', 'overall', 'yes', 'can', 'for', 'per', 'and',
            'the', 'wt', 'oz', 'of', 'or', 'a', 'do', 'an',
            'any', 'am', 'all', 'after', 'again', 'above', 'by',
            'should', 'did', 'each', 'had', 'from', 'her', 'him',
            'but', 'by', 'below', 'between', 'at', 'such', 'some',
            'same', 'out', 'with', 'and', 'is', 'so', 'staples'
        ])
        text_series_clean = text_series.str.lower()
        text_series_clean = text_series_clean.str.replace(r'[^a-z\s]', '', regex=True)  # Remove non-alphabetic characters
        text_series_clean = text_series_clean.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 2 and w not in stopwords]))
        return text_series_clean

    def get_unique_words(df: pd.DataFrame, clean_text_col: str) -> list:
        """Returns unique words from the clean text column."""
        all_words = df[clean_text_col].str.split().explode().unique()
        return list(set(all_words))

    def first_element(text: str) -> str:
        """Extracts the first meaningful part of a text (for vendor-specific processing)."""
        return text.split(', ')[0].split(' - ')[0].split('---')[0] if isinstance(text, str) else ''

    def get_short_text_by_vendor(df: pd.DataFrame) -> pd.DataFrame:
        """Extracts the relevant short text for each vendor."""
        df['text_clean'] = df['Purchase Document Item Short Text'].copy()  # Default to short text

        for i in df.index:
            long_text = df.loc[i, 'Long Text']  # Use the correct column name

            if pd.isna(long_text):  # If Long Text is NaN, use Short Text
                df.loc[i, 'text_clean'] = df.loc[i, 'Purchase Document Item Short Text']
            elif df.loc[i, 'Vendor NM'] == 'GRAINGER INC':
                df.loc[i, 'text_clean'] = first_element(long_text)
            elif df.loc[i, 'Vendor NM'] == 'AMAZON.COM SERVICES INC':
                df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
            elif df.loc[i, 'Vendor NM'] == 'STAPLES ADVANTAGE':
                df.loc[i, 'text_clean'] = str(long_text).split(', ')[0]
                product_text = str(long_text).split(', ')[0]
                df.loc[i, 'text_clean'] = product_text.split('---')[0]
        
        return df


    def extract_nouns_tfidf(text_series: pd.Series) -> list:
        """Uses TF-IDF Vectorizer to extract key words for each row (approximate nouns)."""
        vectorizer = TfidfVectorizer(stop_words="english", max_features=min(len(text_series), 1000))  # Adjust feature limit dynamically
        X = vectorizer.fit_transform(text_series)

        # Convert TF-IDF matrix back to text
        feature_names = vectorizer.get_feature_names_out()

        # Extract top words per row
        word_lists = []
        for row in X.toarray():
            top_indices = row.argsort()[-5:][::-1]  # Get top 5 words per row
            top_words = [feature_names[i] for i in top_indices if row[i] > 0]
            word_lists.append(" ".join(top_words))

        return word_lists

    def extract_noun_words(df: pd.DataFrame, text_col: str) -> pd.DataFrame:
        """Extracts top words using TF-IDF as a proxy for important words."""
        df['noun_words'] = extract_nouns_tfidf(df[text_col])
        return df

    def get_noun_words_reduced(df: pd.DataFrame):
        """Filters clean text to retain only top 5 extracted nouns."""
        df['text_clean'] = df['text_clean'].str.split()
        df['noun_words'] = df['noun_words'].str.split()
        
        df['noun_words_reduced'] = df.apply(lambda row: ' '.join([w for w in row['text_clean'][:5] if w in row['noun_words']]), axis=1)
        
        df['text_clean'] = df['text_clean'].str.join(' ')
        df['noun_words'] = df['noun_words'].str.join(' ')
        
        return df

    def text_parallel_clean(array):
        """Parallel processing wrapper for extract_nouns_tfidf."""
        result = Parallel(n_jobs=8, backend="multiprocessing")(delayed(extract_nouns_tfidf)(text) for text in array)
        return result

    def series_of_lists_to_list_of_strings(series_of_lists: pd.Series) -> list:
        """Converts lists in a Pandas series to a list of joined strings."""
        return series_of_lists.apply(lambda x: ' '.join(x) if isinstance(x, list) else '')

    def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
        """Formats column names for consistency."""
        df.columns = df.columns.str.replace('_', ' ').str.title()
        return df
    
    def get_word_categories(df: pd.DataFrame, use_annotations: bool=False, df_labels=None) -> pd.DataFrame:
        """Reduced noun_words will be split into common_words and rare_words.
        A unique word count is given to each record.

        Args:
            df (pd.DataFrame): dataframe with NOUN_WORDS_REDUCED column
            use_annotations (bool, optional): When use_annotations=True, then if red_flag = 1,
                then the words in the record are not considered in the corpus.
                Defaults to False.
            df_labels (pd.DataFrame, optional): df_labels data in s3. Defaults to None to pull labels from DynamoDB table.
        """
#     start_date = df['purchase_doc_date'].max() - pd.Timedelta(days=450)
#     df = df[df['purchase_doc_date']>start_date].copy()
#     df = df.merge(df_labels, on='tdname', how='left')
        df.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                        col.replace(' ', "_")).upper() for col in df.columns]

        df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].replace(np.nan, 'blank')
        df['NOUN_WORDS_REDUCED'] = df['NOUN_WORDS_REDUCED'].str.split()

        dff = pd.DataFrame()
        for plant in df['PLANT_DESC'].unique():
            print(f'Processing {plant}')

            df_plant = df[df['PLANT_DESC']==plant].copy()

            all_text = []
            if use_annotations:
                for i in df_plant[df_plant['annotation']!=1]['NOUN_WORDS_REDUCED']:
                    all_text.extend(i)
            else:
                for i in df_plant['NOUN_WORDS_REDUCED']:
                    all_text.extend(i)
                
            ###################
            text_series = pd.Series(all_text)
            rare_words = list(text_series.value_counts()[text_series.value_counts()<=3].index)
            rare_words_list = []
            common_words_list = []
            unique_count = []
            for ind in df_plant['NOUN_WORDS_REDUCED'].index:
                unique_counter = 0
                rare_words_found = []
                common_words_found = []
                for w in df_plant.loc[ind, 'NOUN_WORDS_REDUCED']:
                    if w in rare_words:
                        unique_counter+=1
                        rare_words_found.append(w)
                    else:
                        common_words_found.append(w)
                rw = ' '.join(rare_words_found)
                cw = ' '.join(common_words_found)
                rare_words_list.append(rw)
                common_words_list.append(cw)
                unique_count.append(unique_counter)
                
            df_plant['unique_count'] = unique_count
            df_plant['common_words'] = common_words_list
            df_plant['rare_words'] = rare_words_list
            df_plant.columns = df_plant.columns.str.upper()
            df_plant['NOUN_WORDS_REDUCED'] = df_plant['NOUN_WORDS_REDUCED'].apply(lambda x: ' '.join(x))

            dff = pd.concat([dff, df_plant], axis=0)
        return dff
    
    
    df = process_data()
    df = get_short_text_by_vendor(df)
    df['text_clean'] = clean_text(df['text_clean'])
    df = extract_noun_words(df, "text_clean")
    df = get_noun_words_reduced(df)
    df = clean_cols(df)
    df = get_word_categories(df)
    # Ensure 'Purchase Doc Date' is in datetime format
    df['PURCHASE_DOC_DATE'] = pd.to_datetime(df['PURCHASE_DOC_DATE'], errors='coerce')

    # Sort by 'Purchase Doc Date' in ascending order (oldest to newest)
    df = df.sort_values(by='PURCHASE_DOC_DATE', ascending=True).reset_index(drop=True)
    df.columns = [re.sub(r'[^a-zA-Z0-9]', "_",\
                        col.replace(' ', "_")).upper() for col in df.columns]
    session.create_dataframe(df).write.mode("overwrite").save_as_table("PURCHASES_ANOMALY_RESULTS")
    df_last_week = df.groupby('PLANT_DESC').apply(lambda x: x.nlargest(7, 'PURCHASE_DOC_DATE')).reset_index(drop=True)
    session.create_dataframe(df_last_week).write.mode("overwrite").save_as_table("PURCHASES_ANOMALY_RESULTS_LAST_WEEK")
    
    return "PROCESSING DONE"
    

In [31]:
Database = "DS_D_WORKSPACE"
Schema= "PURCHASING_ANOMALY"  # purchasing anomaly schema
session.use_database(Database)
session.use_schema(Schema)

session.sproc.register(func=purchasing_anomaly,
                       name='purchasing_anomaly',
                       stage_location='@STAGE_PURCHASING_ANOMALY',
                       packages=['snowflake-snowpark-python','pandas','numpy','snowflake-ml-python', 'scikit-learn'],
                       is_permanent=True,
                       replace=True,
                       session=session,
                       execute_as='caller')

The version of package 'snowflake-snowpark-python' in the local environment is 1.27.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-ml-python' in the local environment is 1.7.4, which does not fit the criteria for the requirement 'snowflake-ml-python'. Your UDF might not work when the package version is different between the server and your local environment.


In [32]:
session.call('PURCHASING_ANOMALY')

'PROCESSING DONE'